# Filip Twardy MOwNiT lab 6

Wyszukiwarka artykułów

### Dodatkowa informacja

Aby dodatkowo sprawdziź jaki czas dana wyszukiwarka potrzebuje należy na początku komórki dodać wywołanie **%%time**

### Przygotowanie danych

poniżej znajduje się autorski skrypt napisany w **bashu**, który generuje nam 1000 losowych artykułów z wikipedii, wystarczy zmienić wartość w pętli for aby zwiększyć ilość dokumentów. W tym notebooku użyłem 1000 ponieważ mój internet nie pozwala na wygenerowanie więcej w sensownym czasie (1000 dokumentów w około godzine).

```bash
mkdir articles || echo "Articles folder exists"
mkdir bags || echo "Bags folder exists"
for i in {1..1000}
do
    article=$(lynx --dump https://en.wikipedia.org/wiki/Special:Random \
          tail -n +2 \
          sed '/^References/ q' \
          tr "\n%^&*()[]{},." " " \
          tr -s " ")
 
    topic=$(echo $article | awk '{print $1" "$2;}')
    echo $article > ./articles/article$i.txt
    echo "Proceed $i articles with current topic $topic"
done
```

### Słownik słów kluczowych i wektor cech ***bag-of-words***

##### Importy

In [142]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import scipy.sparse as sparse

from collections import Counter, defaultdict
import string
import os
from pprint import pprint as pp

BAGS_FOLDER = "./bags/"
ARTICLES_FOLDER = "./articles/"

log_article = lambda x: f"Artykuł o nazwie -> {x[:-4]}"

try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download('stopwords')

##### Zapisywanie i odczytywanie ***bag-of-words*** z pliku

In [7]:
def save_bag_o_w(file_name, bag_of_words, text=""):
    
    text = "\n".join([" ".join(list(map(str, item))) for item in bag_of_words.items()])
    path = BAGS_FOLDER + file_name
    
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)

        
def load_bag_o_w(file_name):
    
    path = BAGS_FOLDER + file_name
    bag_of_words = Counter()
    
    with open(path, "r", encoding="utf8") as f:
        for line in f.readlines():
            key, val = line.replace("\n", "").split(" ")
            bag_of_words[key] = val
              
    return bag_of_words

##### Funkcja generująca ***bag-of-words***

Funkcja generuje wektor na podstawie podanego teksu, zamienia wszystkie litery na małe, a następnie eliminuje wszystkie niepotrzebne słowa:
- **english_stop_words** słowa, które w języku angielskim nie mają konkretnego znaczenia
- **długość słowa** musi być większa niż 2

Całość zapisujemy w strukturze **Counter** z modułu **collections**

In [8]:
def get_bag_of_words(text):
    
    english_stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()

    text = text.lower() # wielkość liter nie ma znaczenia

    bag_of_words = Counter([ # Couter zamienia liste na slownik wystąpień danego słowa
        stemmer.stem(word) for sentence in sent_tokenize(text) # dzieli tekst na sentencje
                           for word in word_tokenize(sentence) # dzieli sentencje na slowa
                               if word not in english_stop_words 
                                   and len(word) > 2 # usuwa niepotrzebne slowa
    ])
    
    return bag_of_words
    

##### Przykładowe użycie

In [132]:
bag_of_words = get_bag_of_words("The usual dictionary methods are available for Counter objects except for two which work differently for counters.")
pp(bag_of_words)
save_bag_o_w("test.txt", bag_of_words)

load_bag_o_w("test.txt")


Counter({'counter': 2,
         'usual': 1,
         'dictionari': 1,
         'method': 1,
         'avail': 1,
         'object': 1,
         'except': 1,
         'two': 1,
         'work': 1,
         'differ': 1})


Counter({'usual': '1',
         'dictionari': '1',
         'method': '1',
         'avail': '1',
         'counter': '2',
         'object': '1',
         'except': '1',
         'two': '1',
         'work': '1',
         'differ': '1'})

##### Funkcja przetwarzająca wszystkie artykuły

Funkcja przetważa każdy artykuł z folderu ***articles*** oraz tworzy słownik ***base*** wszystkich słów.

In [9]:
def process_articles():
    base = Counter()
    
    _, _, articles = next(os.walk(ARTICLES_FOLDER))
    N = len(articles)
    get_file_name = lambda x: f"article{x}.txt"
    
    for i in range(1, N+1):
        
        file_name = get_file_name(i)
        path = ARTICLES_FOLDER + file_name
    
        with open(path, "r", encoding="utf8") as f:

            bag_of_words = get_bag_of_words(text=f.read())
            base += bag_of_words
            
            save_bag_o_w(file_name=file_name,
                         bag_of_words=bag_of_words)
    
    save_bag_o_w(file_name="base", bag_of_words=base)

### Funkcja generująca macierz ***term-by-document***

In [4]:
def get_term_by_document_matrix(base_dict = "base"):
    
    terms = list(load_bag_o_w(base_dict))
    
    _, _, articles = next(os.walk(ARTICLES_FOLDER))
    N, M = len(articles), len(terms)
    
    get_file_name = lambda x: f"article{x}.txt"
    
    tbd_matrix = sparse.lil_matrix((M, N))
    
    for i in range(N):
        file_name = get_file_name(i+1)

        bag_of_words = load_bag_o_w(file_name=file_name)
        
        for j, term in enumerate(terms):
            tbd_matrix[j, i] = bag_of_words[term]
            
    return tbd_matrix.tocsr(), terms

### Przetworzenie macierzy ***term-by-document*** metodą ***inverse document frequency***

In [42]:
def IDF_one_row(row):
    N = row.shape[1]
    n_w = row.count_nonzero()
    return np.log(N / n_w)

def IDF(tbd_matrix):
    N = tbd_matrix.shape[0]
    idf_matrix = np.zeros((N))
    for i in range(N):
        idf_matrix[i] = IDF_one_row(tbd_matrix[i,:])
        tbd_matrix[i,:] *= idf_matrix[i]
    return tbd_matrix, idf_matrix

### Wstępne przetworzenie

Czas połączyć wszystko w całość, przetwarzamy wszystkie artykuły, ***bag-of-words*** każdego artykułu ląduje w folderze ***bags***. Następnie tworzymy macierz ***term-by-document*** i przetwarzamy ją metodą ***IDF***

In [133]:
process_articles()

In [134]:
unprocessed_matrix, terms = get_term_by_document_matrix()
idf_processed_matrix, idf_matrix = IDF(unprocessed_matrix)

##### Prosta wyszukiwarka

Pierwsza wyszukiwarka bazuje na korelacji między dwoma wektorami danej wzorem:
    
## $ cos(\theta_j) = \frac{q^T d_j}{||q||\cdot||d_j||} = \frac{q^T Ae_j}{||q||\cdot||Ae_j||} $

Wprowadzamy zapytanie **query** oraz liczbę **k** ile tekstów ma wyszukać w kolejności od najtrafniejszego

In [111]:
def find_similar_articles_vanilla(query, tbd_matrix, k=20, base_dict="base", idf_matrix=idf_matrix):
    
    terms = list(load_bag_o_w(base_dict))
    _, _, articles = next(os.walk(ARTICLES_FOLDER))
    N, M = len(articles), len(terms)
    
    get_file_name = lambda x: f"article{x}.txt"
    articles = [get_file_name(x+1) for x in range(N)]
    
    q_vec = sparse.lil_matrix((M, 1))
    q_bag = get_bag_of_words(query)

    for i in range(M):
        q_vec[i, 0] = q_bag[terms[i]] * idf_matrix[i]

        
    
    q_norm =  sparse.linalg.norm(q_vec)
    q_T = q_vec.transpose()
    
    if q_norm == 0:
        return []
    
    ans = []
    
    for j in range(N):
        d_j = tbd_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        
        numerator = (q_T * d_j)[0,0] 
        denominator = (q_norm * d_j_norm)
        
        p = (numerator / denominator, articles[j])
        ans.append(p)
    
    key_func = lambda x : -x[0]
    ans_k = sorted(ans, key=key_func)[:k]
    ans_k = list(map(lambda x: x[1], ans_k))
    
    return ans_k

##### Przykładowe działanie

Wyszukamy **5** artykułów o treści z gier fifa

In [150]:
query = "fifa video game"
k = 5

similar_articles = find_simillar_articles_vanilla(query=query,
                                                  tbd_matrix=idf_processed_matrix,
                                                  idf_matrix=idf_matrix,
                                                  k=k)
print(f"""Wyszukiwarka Prosta 
{"-" * 25}
Szukaj -> '{query}'???
Znaleziono ->
""")
print("\n".join(map(log_article, similar_articles)))

Wyszukiwarka Prosta 
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->

Artykuł o nazwie -> article402
Artykuł o nazwie -> article591
Artykuł o nazwie -> article454
Artykuł o nazwie -> article530
Artykuł o nazwie -> article751


### Wyszukiwarka normalizująca

Druga wyszukiwarka wykorzystuje ten sam wzór ale dodatkowo **normalizuje wektory**

In [94]:
def find_similar_articles_with_norm(query, tbd_matrix, k=20, base_dict="base", idf_matrix=idf_matrix):
    
    terms = list(load_bag_o_w(base_dict))
    _, _, articles = next(os.walk(ARTICLES_FOLDER))
    N, M = len(articles), len(terms)
    
    get_file_name = lambda x: f"article{x}.txt"
    articles = [get_file_name(x+1) for x in range(N)]
    
    q_vec = sparse.lil_matrix((M, 1))
    q_bag = get_bag_of_words(query)

    for i in range(M):
        q_vec[i, 0] = q_bag[terms[i]] * idf_matrix[i]

    for i in range(N):
        norm = sparse.linalg.norm(tbd_matrix[:,i])
        tbd_matrix[:,i] /= norm
    
    q_norm =  sparse.linalg.norm(q_vec)
    q_T = q_vec.transpose()
    q_T /= q_norm
    
    if q_norm == 0:
        return []
    
    ans = []
    
    for j in range(N):
        d_j = tbd_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        
        numerator = (q_T * d_j)[0,0] 
        denominator = (q_norm * d_j_norm)
        
        p = (numerator / denominator, articles[j])
        ans.append(p)
    
    key_func = lambda x : -x[0]
    ans_k = sorted(ans, key=key_func)[:k]
    ans_k = list(map(lambda x: x[1], ans_k))
    
    return ans_k

##### Przykładowe działanie

Wyszukamy **5** artykułów o treści z gier fifa

In [151]:
query = "fifa video game"
k = 5

similar_articles = find_similar_articles_with_norm(query=query,
                                                  tbd_matrix=idf_processed_matrix,
                                                  idf_matrix=idf_matrix,
                                                  k=k)

print(f"""Wyszukiwarka Normalizująca 
{"-" * 25}
Szukaj -> '{query}'???
Znaleziono ->
""")
print("\n".join(map(log_article, similar_articles)))

Wyszukiwarka Normalizująca 
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->

Artykuł o nazwie -> article402
Artykuł o nazwie -> article591
Artykuł o nazwie -> article454
Artykuł o nazwie -> article530
Artykuł o nazwie -> article751


### Wyszukiwarka SVD

Trzecia wyszukiwarka wykorzystuje algorytm **symulowanego wyżarzania** wraz z **low rank approximation**

In [125]:
def find_similar_articles_with_svd(query, tbd_matrix, k=20,
                                                       svd_k=100,
                                                       base_dict="base",
                                                       idf_matrix=idf_matrix):
    
    terms = list(load_bag_o_w(base_dict))
    _, _, articles = next(os.walk(ARTICLES_FOLDER))
    N, M = len(articles), len(terms)
    
    get_file_name = lambda x: f"article{x}.txt"
    articles = [get_file_name(x+1) for x in range(N)]
    
    q_vec = sparse.lil_matrix((M, 1))
    q_bag = get_bag_of_words(query)

    for i in range(M):
        q_vec[i, 0] = q_bag[terms[i]] * idf_matrix[i]

    
    q_norm =  sparse.linalg.norm(q_vec)
    q_T = q_vec.transpose()
    
    if q_norm == 0:
        return []
    
    U, S, V = sparse.linalg.svds(tbd_matrix, svd_k)

    U_k = sparse.lil_matrix(U[:, :svd_k])
    S_k = sparse.lil_matrix(np.diag(S[:svd_k]))
    V_k = sparse.lil_matrix(V[:svd_k, :])
   
    tbd_matrix_filtered = sparse.csc_matrix(U_k * S_k * V_k)
    
    ans = []
    
    for j in range(N):
        d_j = tbd_matrix_filtered[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        
        numerator = (q_T * d_j)[0,0] 
        denominator = (q_norm * d_j_norm)
        
        p = (numerator / denominator, articles[j])
        ans.append(p)
    
    key_func = lambda x : -x[0]
    ans_k = sorted(ans, key=key_func)[:k]
    ans_k = list(map(lambda x: x[1], ans_k))
    
    return ans_k

### Przykładowe użycie 

Przykładowe użycia zostaną dokonane dla różnych wartości **k** dla **low rank approximation**
- $k = 20, 50, 100, 200, 400$

In [154]:
query = "fifa video game"
K = [20, 50, 100 ,200 ,400]

for k in K:
    similar_articles = find_similar_articles_with_svd(query=query,
                                                      tbd_matrix=idf_processed_matrix,
                                                      idf_matrix=idf_matrix,
                                                      k=5,
                                                      svd_k=k)
    print(f"""{"-" * 25}
Wyszukiwarka SVD dla k={k} 
{"-" * 25}
Szukaj -> '{query}'???
Znaleziono ->
    """)
    print("\n".join(map(log_article, similar_articles)))

-------------------------
Wyszukiwarka SVD dla k=20 
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->
    
Artykuł o nazwie -> article128
Artykuł o nazwie -> article327
Artykuł o nazwie -> article930
Artykuł o nazwie -> article23
Artykuł o nazwie -> article835
-------------------------
Wyszukiwarka SVD dla k=50 
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->
    
Artykuł o nazwie -> article751
Artykuł o nazwie -> article892
Artykuł o nazwie -> article933
Artykuł o nazwie -> article128
Artykuł o nazwie -> article990
-------------------------
Wyszukiwarka SVD dla k=100 
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->
    
Artykuł o nazwie -> article591
Artykuł o nazwie -> article933
Artykuł o nazwie -> article751
Artykuł o nazwie -> article892
Artykuł o nazwie -> article916
-------------------------
Wyszukiwarka SVD dla k=200 
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->
    
Artykuł o nazwie -

### Podsumowanie

Wyszukiwarka wykorzystująca ***IDF*** oraz ***symulowane wyżarzanie*** najlepiej radzi sobie dla ***K=100***
co pozwala nam znacząco zmniejszyć potrzebną pamięć na przetrzymywanie macierzy ***term-by-document***.

Poniżej dokonam dodatkowo prezentacji jak algorytm działa bez preporcessingu **IDF**

In [155]:
query = "fifa video game"
k = 5

similar_articles = find_simillar_articles_vanilla(query=query,
                                                  tbd_matrix=unprocessed_matrix, # <- tutaj
                                                  idf_matrix=idf_matrix,
                                                  k=k)
print(f"""Wyszukiwarka Prosta bez IDF
{"-" * 25}
Szukaj -> '{query}'???
Znaleziono ->
""")
print("\n".join(map(log_article, similar_articles)))

Wyszukiwarka Prosta bez IDF
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->

Artykuł o nazwie -> article402
Artykuł o nazwie -> article591
Artykuł o nazwie -> article454
Artykuł o nazwie -> article530
Artykuł o nazwie -> article751


In [156]:
query = "fifa video game"
k = 5

similar_articles = find_similar_articles_with_norm(query=query,
                                                  tbd_matrix=unprocessed_matrix, # <- tutaj
                                                  idf_matrix=idf_matrix,
                                                  k=k)

print(f"""Wyszukiwarka Normalizująca bez IDF
{"-" * 25}
Szukaj -> '{query}'???
Znaleziono ->
""")
print("\n".join(map(log_article, similar_articles)))

Wyszukiwarka Normalizująca bez IDF
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->

Artykuł o nazwie -> article402
Artykuł o nazwie -> article591
Artykuł o nazwie -> article454
Artykuł o nazwie -> article530
Artykuł o nazwie -> article751


In [159]:
query = "fifa video game"
k = 100

similar_articles = find_similar_articles_with_svd(query=query,
                                                  tbd_matrix=unprocessed_matrix, # <- tutaj
                                                  idf_matrix=idf_matrix,
                                                  k=5,
                                                  svd_k=k)
print(f"""{"-" * 25}
Wyszukiwarka SVD dla k={k} bez IDF
{"-" * 25}
Szukaj -> '{query}'???
Znaleziono ->
    """)
print("\n".join(map(log_article, similar_articles)))

-------------------------
Wyszukiwarka SVD dla k=100 bez IDF
-------------------------
Szukaj -> 'fifa video game'???
Znaleziono ->
    
Artykuł o nazwie -> article591
Artykuł o nazwie -> article933
Artykuł o nazwie -> article751
Artykuł o nazwie -> article892
Artykuł o nazwie -> article916
